In [1]:
import pandas as pd
import numpy as np
import time
from math import *
from src.display import horizontal_cluster
from src.find_HC import *

# Test of the display function

In [11]:
start = time.time()
horizontal_cluster('AIQTSDEH...KV..ATPANW....RP..................GDK.V.VVP.PP.NTQE.MA..EERMKEG')
end = time.time()
print('time :', end-start)

[['-' 'A' '▣' 'K' '★' 'R' 'K' '★' '☐' 'A' 'M']
 ['-' 'I' 'D' 'V' 'A' '★' 'V' '★' 'Q' 'E' 'K']
 ['-' 'Q' 'E' 'A' 'N' '⯁' 'V' '★' 'E' 'E' 'E']
 ['-' '☐' 'H' '☐' 'W' 'D' 'V' 'N' 'M' 'R' '⯁']
 ['A' '▣' 'K' '★' 'R' 'K' '★' '☐' 'A' 'M' '-']
 ['I' 'D' 'V' 'A' '★' 'V' '★' 'Q' 'E' 'K' '-']
 ['Q' 'E' 'A' 'N' '⯁' 'V' '★' 'E' 'E' 'E' '-']
 ['☐' 'H' '☐' 'W' 'D' 'V' 'N' 'M' 'R' '⯁' '-']]
time : 0.001237630844116211


# Find HCs of a sequence

In [5]:
#seq='AIQTSDEH...KV..ATPANW....RP..................GDK.V.VVP.PP.NTQE.MA..EERMKEG'
seq='-----....--...----.------...............---...---.---............-----....--.-....-.-..-.--.......--..-...-.-..-.-..-.-.-........................--.-.-...-.--...GQM............PRL..E..VV.Q....T..L.T..A.G...Y.....E....D....V....-L....P....L.L...P.D..........Q.....-....VT.......L............C......N.......A....A.......G.....L......H........D.....T...S....TA..ELAvalalasgrhldwfardqvtgtwgqrpgraladqrvlivgygrigaaiearlqgfevasvtrvsgrgrsgpplvhpvgdlhgllpe..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................advvfvvaphtpqtegmfgarelgllpdgallvnvargklvdtaallaelsarritaaldvtdpeplpvghplwhapgvlisphvgGA..SSAFT.P..R.A.D.RLI.AAQLT.R.FA.AG......Q.PLANVV-'
hc=binary_coding(seq)
hc

{(161, 163): '001',
 (178, 228): '101100100010011',
 (238, 240): '11',
 (268, 277): '101',
 (360, 366): '1010101',
 (372, 375): '1011',
 (380, 384): '10001',
 (397, 424): '1111010010001000100101001001',
 (434, 436): '110',
 (438, 445): '10010011',
 (1236, 1240): '11111',
 (1250, 1251): '11',
 (1256, 1259): '1011',
 (1264, 1268): '11101',
 (1273, 1274): '11',
 (1279, 1283): '11001',
 (1288, 1294): '1000101',
 (1302, 1304): '100',
 (1306, 1307): '11',
 (1312, 1315): '1110',
 (1340, 1351): '110001001',
 (1365, 1369): '10011'}

# Running on the part of the database

In [12]:
start = time.time()
all_HCs = get_analyse('part_of_data.txt', 'soluble_domains_pfam_accession.txt')
end = time.time()
print('time :', end-start)

time : 2.134244203567505


In [4]:
all_HCs

{3: 35039,
 5: 26501,
 9: 17102,
 17: 11768,
 11: 10599,
 7: 9177,
 13: 8681,
 21: 7789,
 25: 5353,
 27: 4987,
 19: 4437,
 85: 4030,
 29: 3839,
 35: 3566,
 137: 3430,
 15: 3159,
 89: 3053,
 49: 2959,
 145: 2922,
 73: 2822,
 341: 2414,
 59: 2355,
 61: 2305,
 177: 2304,
 69: 2291,
 433: 2149,
 81: 1955,
 41: 1952,
 141: 1910,
 283: 1738,
 23: 1704,
 43: 1678,
 53: 1625,
 139: 1451,
 273: 1406,
 217: 1385,
 47: 1309,
 197: 1305,
 171: 1291,
 1097: 1285,
 153: 1284,
 87: 1276,
 169: 1269,
 291: 1255,
 683: 1241,
 395: 1234,
 709: 1214,
 83: 1207,
 209: 1197,
 189: 1183,
 1419: 1183,
 1733: 1182,
 945: 1179,
 175: 1169,
 3467: 1167,
 37: 1164,
 343: 1163,
 1365: 1146,
 1367: 1144,
 701: 1142,
 2731: 1141,
 99: 1128,
 425: 1126,
 687: 1118,
 3781: 1115,
 123: 1114,
 2735: 1108,
 7563: 1108,
 339: 1106,
 2749: 1105,
 1165: 1101,
 10941: 1099,
 851: 1075,
 5463: 1068,
 2331: 1068,
 10927: 1056,
 43709: 1053,
 2193: 1015,
 465: 1008,
 51: 996,
 379: 993,
 1403: 975,
 977: 954,
 4387: 953,
 5499

## Comparison with the csv file of Lamiable

In [13]:
df = pd.read_csv('files_n_articles/HCDB_HC-counts_secondary-struct_sorted.csv', sep=',')
true_HCs = {}
for hc in range(len(df)):
    true_HCs[df['#Peitsch'][hc]]=df['Effectif'][hc]
print('number of HCs in the csv file :', len(true_HCs))

number of HCs in the csv file : 476


In [7]:
print('total effectif of HCs with our HC detector :', sum(all_HCs.values()))
print('total effectif of HCs in the csv file :', sum(true_HCs.values()))

total effectif of HCs with our HC detector : 338813
total effectif of HCs in the csv file : 187298


In [8]:
counter = 0
for hc in all_HCs:
    if hc in true_HCs:
        counter += 1
print('number of common HCs in both dictionary :', counter)

number of common HCs in both dictionary : 262


# Running on the whole database

In [12]:
start = time.time()
database_all_HCs = get_analyse('../Pfam-A.rp15', 'soluble_domains_pfam_accession.txt')
end = time.time()
print('time :', end-start)

time : 4171.719877719879


In [ ]:
database_all_HCs

## Comparison with the csv file of Lamiable

In [51]:
counter = 0
for hc in database_all_HCs:
    if hc in true_HCs:
        counter += 1
print('number of common HCs in both dictionary :', counter)

number of common HCs in both dictionary : 417
